In [4]:
import numpy as np
from math import sqrt
def conjugate_gradient(A, b, x, eps = 1.e-4):
    r    = b - A @ x # residue
    d    = r # direction
    rTr   = r.T @ r
    for i in range(A.shape[0]):
        Ad       = A @ d
        alpha    = rTr / (d.T @ Ad)
        x        = x + alpha * d
        r        = r - alpha * Ad
        rTr_prev = rTr
        rTr = r.T @ r
        #print('err : ', sqrt(rTr))
        if sqrt(rTr) <= eps: return x
        beta   = rTr / rTr_prev
        d      = r + beta * d
    return x

def test_cg():
    A=np.array([[1, -1], [-1, 2]])
    b=np.array([1, 0])
    print(conjugate_gradient(A, b, np.array([0.1, 0.1])))

    for N in [10, 100, 1000, 10000]:
        a = np.random.rand(N, N)
        A = np.dot(a, a.T) + 0.1 * np.eye(N)
        b = np.random.rand(N,1)

        print(np.all(A==A.T))
        import time
        print('solving')
        t = time.time()
        sol = conjugate_gradient(A, b, np.zeros((N, 1)))
        print('runtime :', time.time() - t)
        t = time.time()
        sol1 = np.linalg.solve(A, b)
        print('runtime :', time.time() - t)
        s = sol-sol1
        print(sqrt(s.T @ s))


In [ ]:
import LEFDEFParser
import numpy
l = LEFDEFParser.LEFReader()
d = LEFDEFParser.DEFReader()
l.readLEF('sample.lef')
d.readDEF('sample.def')
bbox = d.bbox()

class Vertex:
    def __init__(self, c):
        self._comp = c
        self._nbrs = []
        self._pinNbrs = []
    def __repr__(self):
        return self._comp.name() + f' {self._nbrs} {self._pinNbrs}'
        
V = [Vertex(c) for c in d.components()]
Vdict = {V[i]._comp.name():i for i in range(len(V))}

P = [(p.name(), p.origin()) for p in d.pins()]
Pdict = {P[i][0]:i for i in range(len(P))}

for n in d.nets():
    u = list()
    pins = list()
    for p in n.pins():
        if p[0] != 'PIN':
            assert(p[0] in Vdict)
            u.append(Vdict[p[0]])
        else:
            assert(p[1] in Pdict)
            pins.append(Pdict[p[1]])
    for i in range(len(u)):
        for p in pins:
          V[u[i]]._pinNbrs.append(p)
        for j in range(i + 1, len(u)):
            V[u[i]]._nbrs.append(u[j])
            V[u[j]]._nbrs.append(u[i])

A = np.zeros((len(V), len(V)))
bx = np.zeros((len(V), 1))
by = np.zeros((len(V), 1))
for i in range(len(V)):
    A[i][i] += (len(V[i]._nbrs) + len(V[i]._pinNbrs))*1.
    for nbr in V[i]._nbrs:
        A[i][nbr] -= 1
    for p in V[i]._pinNbrs:
        bx[i] += P[p][1].x
        by[i] += P[p][1].y
print(A)
print('solving')
t = time.time()
solx = np.linalg.solve(A, bx)
soly = np.linalg.solve(A, by)
print('runtime :', time.time() - t)
#print(V[0])
#print(V[59])
#print(len(V))
